In [1]:
import pandas as pd
import re

In [3]:
df = pd.read_csv('hdf_service_20230117_sentences_with labels.csv', sep='\t', encoding='utf_8_sig', low_memory=True)
df

,service_id,all_doctor_text,信息支持1-病情描述/分析/诊断,信息支持2-科普读物/其他病人经历,id,信息支持3-线上/门诊或住院流程/时间/费用,信息支持4-治疗建议,信息支持,情感支持
0,8508082035,['请你下周五早上到华山医院虹桥院区门诊找我加号',0,0,1,1,0,1,0
1,8508082035,'好的 第1个问题，拔牙没有任何关系可以拔的 第2个问题，可以口服强的松每天早上6粒',0,0,2,0,1,1,0
2,8508082035,'一个月左右如果病情比较稳定，就改为一天6粒一天4粒',0,0,3,0,1,1,0
3,8508082035,'我的门诊时间： 周一下午和周五早上（华山医院虹桥院区门诊1楼7号房间） 周二上午（华山医院...,0,0,4,1,0,1,0
4,8508082035,'赵重波 通知：FcRn 单抗HBM 9161治疗全身型重症肌无力临床研究华山入组的8例受试...,1,0,5,0,0,1,0
...,...,...,...,...,...,...,...,...,...
3065544,8887571478,['明天早上不吃不喝，早上就给您手术'],0,0,3065545,0,0,0,0
3065545,8887571559,['您好',0,0,3065546,0,0,0,0
3065546,8887571559,'根据您的检查结果目前诊断明确，建议手术治疗，手术范围至少右侧甲状腺切除加右侧气管旁和气管前...,1,0,3065547,0,1,1,0
3065547,8887571865,['胆囊癌，肝肾功能都不太好 如果不达标，病人生存＜半年'],1,0,3065548,0,0,1,0


In [23]:
df = pd.read_csv('hdf_service_20230117.csv', sep='\t', encoding='utf_8_sig', low_memory=True)
df

/home/anqizhao/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (44,45,46,48) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,doctor_id,service_id,service_list,service_list_time,date_scraped,patient_age,patient_gender,title,service_type,processed_service_type,...,time_specific,pre_consult,pre_doctor,pre_hospital,pre_keshi,shanghai,pre_doc_title,hospital_三甲,pre_doc_hospital_三甲,coherence_score
0,610571686,8508080911,6478997403,20200602,20220603,31岁,女,视神经脊髓炎门诊患者目前：已开检查,扫码报到,扫码报到,...,20200602 06:39,0,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
1,243591,8508081687,6478998064,20200602,20220604,40岁,女,外导管内乳头状瘤门诊患者目前：已开检查,扫码报到,扫码报到,...,20200602 07:56,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0.911853
2,259551,8508082035,6478998382,20200602,20220603,28岁,女,目前每天犯困疲乏，现在每天强的松4粒，一粒溴吡斯的明，硫唑嘌呤片一粒，奥美拉唑1粒门诊患者目...,扫码报到,扫码报到,...,20200602 08:10,0,NaN,NaN,NaN,1,NaN,NaN,NaN,0.895641
3,150812,8508082473,6478998789,20200602,20220603,47岁,男,下肢静脉曲张 已有二十多年了，下肢变色，走路有点无力站立脚有点 已检查过，寻求医生治疗意见-...,加班门诊,加班门诊,...,20200602 08:24,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
4,6964411335,8508083390,6478999609,20200602,20220604,44岁,女,面中部有红斑，有疹子，脸部红，肿，敏感，有结痂，摸着很粗糙，鼻子也红肿，皮肤不耐受。门诊患者...,扫码报到,扫码报到,...,20200602 08:58,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0.928419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2181083,624022760,8887571478,6499544761,20221031,20221111,71岁,女,输尿管结石,扫码报到,扫码报到,...,20221031 22:13,0,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
2181084,551502716,8887571559,6499544832,20221031,20221111,35岁,男,发现甲状腺右叶结节2周。 患者于2022-10-11广西医科大学第一附属医院甲状腺彩超......,电话问诊,电话问诊,...,20221031 22:18,1,陈金水,解放军联勤保障部队第九九一医院,普通/心胸外科,1,副主任医师,NaN,1.0,0.868866
2181085,11388,8887571865,6499545093,20221031,20221111,51岁,男,胆管癌,扫码报到,扫码报到,...,20221031 22:36,0,NaN,NaN,NaN,1,NaN,NaN,NaN,0.900225
2181086,9032049258,8887572274,6499545452,20221031,20221111,72岁,男,肺结核,扫码报到,扫码报到,...,20221031 23:01,0,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN


In [24]:
#(705999, 52)
df[df['shanghai']==1].shape

(705999, 52)

In [27]:
df_shanghai = df[df['shanghai']==1]
df_shanghai

,doctor_id,service_id,service_list,service_list_time,date_scraped,patient_age,patient_gender,title,service_type,processed_service_type,...,time_specific,pre_consult,pre_doctor,pre_hospital,pre_keshi,shanghai,pre_doc_title,hospital_三甲,pre_doc_hospital_三甲,coherence_score
0,610571686,8508080911,6478997403,20200602,20220603,31岁,女,视神经脊髓炎门诊患者目前：已开检查,扫码报到,扫码报到,...,20200602 06:39,0,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
2,259551,8508082035,6478998382,20200602,20220603,28岁,女,目前每天犯困疲乏，现在每天强的松4粒，一粒溴吡斯的明，硫唑嘌呤片一粒，奥美拉唑1粒门诊患者目...,扫码报到,扫码报到,...,20200602 08:10,0,NaN,NaN,NaN,1,NaN,NaN,NaN,0.895641
11,551502716,8508085930,6479001909,20200602,20221017,31岁,女,免疫力较差门诊患者目前：暂时没开三个月后复诊,扫码报到,扫码报到,...,20200602 10:14,0,NaN,NaN,NaN,1,NaN,NaN,NaN,0.920766
16,734554442,8508088017,6479003806,20200602,20221017,37岁,女,癫痫门诊患者目前：暂时没开三个月后复诊,扫码报到,扫码报到,...,20200602 11:08,0,NaN,NaN,NaN,1,NaN,NaN,NaN,0.945336
20,764331918,8508089790,6479005397,20200602,20220604,34岁,女,乳腺炎，有脓门诊患者目前：暂时没开,扫码报到,扫码报到,...,20200602 11:53,0,NaN,NaN,NaN,1,NaN,NaN,NaN,0.928178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2181083,624022760,8887571478,6499544761,20221031,20221111,71岁,女,输尿管结石,扫码报到,扫码报到,...,20221031 22:13,0,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
2181084,551502716,8887571559,6499544832,20221031,20221111,35岁,男,发现甲状腺右叶结节2周。 患者于2022-10-11广西医科大学第一附属医院甲状腺彩超......,电话问诊,电话问诊,...,20221031 22:18,1,陈金水,解放军联勤保障部队第九九一医院,普通/心胸外科,1,副主任医师,NaN,1.0,0.868866
2181085,11388,8887571865,6499545093,20221031,20221111,51岁,男,胆管癌,扫码报到,扫码报到,...,20221031 22:36,0,NaN,NaN,NaN,1,NaN,NaN,NaN,0.900225
2181086,9032049258,8887572274,6499545452,20221031,20221111,72岁,男,肺结核,扫码报到,扫码报到,...,20221031 23:01,0,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN


In [28]:
columns = ['service_id','all_doctor_text']
df1 = df_shanghai[columns].copy()
df1

,service_id,all_doctor_text
0,8508080911,NaN
2,8508082035,请你下周五早上到华山医院虹桥院区门诊找我加号。 好的 第1个问题，拔牙没有任何关系可以拔的 ...
11,8508085930,口服的是胸腺肽肠溶片 一般建议连续服用至少三个月左右，能用半年最好。您可以先再用一个月，根据...
16,8508088017,继续服药 单药奥卡、近一年未发作
20,8508089790,这种溢液和用药没关系，应该以前就有的 很多药物会导致溢乳，比如治疗胃溃疡的很多药物，但外敷药...
...,...,...
2181083,8887571478,明天早上不吃不喝，早上就给您手术。
2181084,8887571559,您好！根据您的检查结果目前诊断明确，建议手术治疗，手术范围至少右侧甲状腺切除加右侧气管旁和气...
2181085,8887571865,？ 胆囊癌，肝肾功能都不太好 如果不达标，病人生存＜半年
2181086,8887572274,好


In [30]:
# 使用逗号、句号、分号、感叹号和问号分割文本
def split_sentences(text):
    punctuation = "。；！？,.!?"
    sentences = []
    sentence = ""
    for char in text:
        if char in punctuation:
            if sentence:
                sentences.append(sentence.strip())
                sentence = ""
        else:
            sentence += char
    if sentence:
        sentences.append(sentence.strip())
    return sentences

In [33]:
# 将 "all_doctor_text" 列中的所有元素转换为字符串
df1['all_doctor_text'] = df1['all_doctor_text'].astype(str)
# 对 "all_doctor_text" 列应用分割函数
df1['all_doctor_text'] = df1['all_doctor_text'].apply(split_sentences)
# 将每个句子拆分成新的行
new_rows = []
for _, row in df1.iterrows():
    for sentence in row['all_doctor_text']:
        new_rows.append({"service_id": row["service_id"], "all_doctor_text": sentence})
# 创建新的 DataFrame
new_df1 = pd.DataFrame(new_rows)

In [34]:
# 打印生成的新 DataFrame
print(new_df1)

         service_id                                    all_doctor_text
0        8508080911                                            ['nan']
1        8508082035                          ['请你下周五早上到华山医院虹桥院区门诊找我加号'
2        8508082035        '好的 第1个问题，拔牙没有任何关系可以拔的 第2个问题，可以口服强的松每天早上6粒'
3        8508082035                        '一个月左右如果病情比较稳定，就改为一天6粒一天4粒'
4        8508082035  '我的门诊时间： 周一下午和周五早上（华山医院虹桥院区门诊1楼7号房间） 周二上午（华山医院...
...             ...                                                ...
3203213  8887571559                                              ['您好'
3203214  8887571559  '根据您的检查结果目前诊断明确，建议手术治疗，手术范围至少右侧甲状腺切除加右侧气管旁和气管前...
3203215  8887571865                     ['胆囊癌，肝肾功能都不太好 如果不达标，病人生存＜半年']
3203216  8887572274                                              ['好']
3203217  8887572288                                            ['nan']

[3203218 rows x 2 columns]


In [37]:
new_df1['service_id'].nunique()

705999

In [46]:
df_cleaned = new_df1[new_df1['all_doctor_text'] != "['nan']"]
df_cleaned

,service_id,all_doctor_text
1,8508082035,['请你下周五早上到华山医院虹桥院区门诊找我加号'
2,8508082035,'好的 第1个问题，拔牙没有任何关系可以拔的 第2个问题，可以口服强的松每天早上6粒'
3,8508082035,'一个月左右如果病情比较稳定，就改为一天6粒一天4粒'
4,8508082035,'我的门诊时间： 周一下午和周五早上（华山医院虹桥院区门诊1楼7号房间） 周二上午（华山医院...
5,8508082035,'赵重波 通知：FcRn 单抗HBM 9161治疗全身型重症肌无力临床研究华山入组的8例受试...
...,...,...
3203212,8887571478,['明天早上不吃不喝，早上就给您手术']
3203213,8887571559,['您好'
3203214,8887571559,'根据您的检查结果目前诊断明确，建议手术治疗，手术范围至少右侧甲状腺切除加右侧气管旁和气管前...
3203215,8887571865,['胆囊癌，肝肾功能都不太好 如果不达标，病人生存＜半年']


In [47]:
df_cleaned['service_id'].nunique()

568330